### 1. 問題の分析

* **競技プログラミング視点での分析**

  * 最速は値→添字のハッシュ（`Map<number, number>`）を使う**1パス法**。各要素 `x` に対し補数 `target - x` を先に見たかを **O(1)** 期待で照会し、見つかれば即返す → **時間 O(n)**。
  * メモリは最悪で n 件の値を保持 → **空間 O(n)**。配列は不変操作（非破壊）で GC 負荷を極小化。

* **業務開発視点での分析**

  * **型安全性**：引数は `number[]`。実装では不変利用（副作用なし）。
  * **保守性/可読性**：`seen`（値→最初の添字）という責務限定の構造、早期 return、例外メッセージの明確化。
  * **エラーハンドリング**：制約外入力（長さ/有限数でない）を検知して `TypeError` / `RangeError` を送出（LeetCode では常に正当入力だが、業務品質では有用）。

* **TypeScript特有の考慮点**

  * **型推論**：ローカル変数は `const`/`let` で自動推論。戻り値は `number[]` を明示。
  * **コンパイル時最適化**：ユニオン/ジェネリクスは不要（数値専用）なので最短経路で最適化を阻害しない。
  * **null安全性**：`Map#get` の戻りを一時変数に取り、`undefined` 判定を単純分岐で処理。

---

### 2. アルゴリズムアプローチ比較

| アプローチ                   | 時間計算量      | 空間計算量 | TS実装コスト | 型安全性 | 可読性 | 備考                    |
| ----------------------- | ---------- | ----- | ------- | ---- | --- | --------------------- |
| 方法A：ハッシュ1パス（値→添字 `Map`） | **O(n)**   | O(n)  | 低       | 高    | 高   | 補数照会→即時 return（最適）    |
| 方法B：ソート＋二ポインタ（元添字保持）    | O(n log n) | O(n)  | 中       | 中    | 中   | 添字復元用の配列が必要           |
| 方法C：二重ループ               | O(n²)      | O(1)  | 低       | 高    | 高   | 小規模のみ妥当、Follow-up 非充足 |

---

### 3. 選択したアプローチと理由

* **選択したアプローチ**: 方法A（ハッシュ1パス）
* **理由**:

  * **計算量**：Follow-up 要件（O(n²) より良い）を満たす **O(n)**。
  * **TypeScript**：`Map<number, number>` が型安全で記述量も最小。
  * **保守性/可読性**：単純な for ループ＋早期 return。副作用なしの Pure 実装。
* **TypeScript特有の最適化ポイント**:

  * 数値専用のためジェネリクスを使わず、型を絞って JIT/最適化を阻害しない。
  * `const` 利用で再代入防止、`readonly` 的運用（配列は不変操作）。

---

### 4. 実装コード（LeetCode 形式・ESM方針 / Node.js v22.14.0, 外部ライブラリ不可）

> 注: LeetCode の TypeScript ランタイムは関数のみを評価します（`export` は不要/非推奨）。本実装は **Pure function** で、ローカル ESM 環境でもそのまま動作します（関数定義のみのためモジュール化は不要）。

```typescript
/**
 * Two Sum — 1パスのハッシュ法（Pure）
 * @param nums 入力配列（整数のみを想定、非破壊で扱う）
 * @param target 目標和
 * @returns 条件を満たす 2 要素の添字（任意順）
 * @throws {TypeError} nums が配列でない / 要素や target が有限数でない場合
 * @throws {RangeError} 配列長が [2, 1e4] を外れる場合（業務品質のための防御的検証）
 * @complexity Time: O(n), Space: O(n)
 */
function twoSum(nums: number[], target: number): number[] {
  // --- 入力検証（LeetCode では前提満たすが、業務品質として保持） ---
  if (!Array.isArray(nums)) {
    throw new TypeError('nums must be an array');
  }
  const n = nums.length;
  if (n < 2 || n > 1e4) {
    throw new RangeError('nums length is out of range [2, 1e4]');
  }
  if (!Number.isFinite(target)) {
    throw new TypeError('target must be a finite number');
  }
  // 要素は有限数であることを軽量チェック
  for (let i = 0; i < n; i++) {
    const v = nums[i];
    if (!Number.isFinite(v)) {
      throw new TypeError(`nums[${i}] must be a finite number`);
    }
  }

  // --- 本処理（値→最初の添字） ---
  const seen = new Map<number, number>(); // value -> index

  for (let i = 0; i < n; i++) {
    const x = nums[i];
    const need = target - x;
    const j = seen.get(need);
    if (j !== undefined) {
      // 一意解前提につき即 return（添字の順不同）
      return [j, i];
    }
    // 同値重複時は最初の出現を固定
    if (!seen.has(x)) seen.set(x, i);
  }

  // 問題設定では必ず見つかるが、防御的に例外より固定値を返すなら下記：
  // return [-1, -1];
  throw new RangeError('No valid two-sum solution found');
}

Analyze Complexity
Runtime 2 ms
Beats 74.43%
Memory 57.05 MB
Beats 48.39%

```

---

## TypeScript固有の最適化観点

* **型安全性の活用**

  * 問題が数値専用のため、敢えてジェネリクスを使わず `number[]` に限定し、呼び出し側の誤用をコンパイル時に排除。
  * 実行時は `Number.isFinite` による軽量ガードでダブルセーフティ。

* **コンパイル時最適化**

  * ループは `for (let i = 0; i < n; i++)` の単純形で分岐予測と最適化が安定。
  * 余計なオブジェクト生成を避け、`Map#get` の戻りを一時変数に保持して分岐（再取得なし）。

* **開発効率と保守性**

  * 副作用なしの **Pure function**、早期 `return` により読みやすくテスト容易。
  * 例外はホットパス外（先頭検証）で発生し、計測パスを汚さない。

**さらに数％の伸び**と**わずかなメモリ削減**を狙うなら、LeetCodeの前提（常に妥当入力）を活かして「防御的チェックを外す」「`Map`→nullプロトタイプの辞書」を試す価値があります。下記2点が要点です。

* **Runtime 改善**: 先頭の型/範囲チェックを外す（LeetCodeは制約前提）。`Map#get/has` より軽い **`Object.create(null)` 辞書 + `in` 判定** に変更。
* **Memory 改善**: `Map` は内部構造のオーバーヘッドがあり、**キーが多いほど辞書（null-proto）有利**になりがち。

---

## 提案：ジャッジ最適化版（TypeScript / ESM方針、LeetCode関数フォーマット）

```typescript
/**
 * Two Sum — Judge-optimized (TS / Pure)
 * @param nums 数値配列（LeetCode前提で妥当入力）
 * @param target 目標和
 * @returns 条件を満たす 2 要素の添字
 * @complexity Time: O(n), Space: O(n)
 */
function twoSum(nums: number[], target: number): number[] {
  // nullプロトタイプ辞書：プロトタイプ汚染なし、キー存在判定が軽い
  const seen: Record<string, number> = Object.create(null);

  // 1パス：補数が既出なら即返す / 未登録なら初出を保持
  // ＊同じ要素を二度使わない条件を満たすため「先に照会→後で登録」
  for (let i = 0, n = nums.length; i < n; i++) {
    const x = nums[i];
    const need = target - x;
    const kNeed = need as unknown as string; // 数値キーは内部的に文字列化

    if (kNeed in seen) {
      return [seen[kNeed], i];
    }

    const kX = x as unknown as string;
    if (!(kX in seen)) {
      seen[kX] = i; // 最初の出現のみ保持（重複値で最左を固定）
    }
  }

  // 問題の前提では解が必ず存在
  return [-1, -1];
}

Analyze Complexity
Runtime 3 ms
Beats 64.32%
Memory 56.44 MB
Beats 71.03%

```

### 変更点と狙い

* **入力検証を削除**：純粋にホットパスのみ。LeetCode前提で安全。
* **`Map`→`Record(null-proto)`**：キー変換は発生しますが、要素数が増えるほど軽量辞書が効くケースが多いです。
* **`in` 判定**：`undefined` 番人値に依存せず、存在チェックを明示・高速化。

---

## 代替：保守性重視（業務コード寄り）

入力検証あり・`Map<number, number>` のまま運用（読みやすさ・意図の明確さ重視）。LeetCodeの計測では若干不利になり得ますが、現場品質としては推奨です。

```typescript
function twoSum(nums: number[], target: number): number[] {
  const n = nums.length;
  const seen = new Map<number, number>();

  for (let i = 0; i < n; i++) {
    const x = nums[i];
    const j = seen.get(target - x);
    if (j !== undefined) return [j, i];
    if (!seen.has(x)) seen.set(x, i);
  }
  return [-1, -1];
}

Analyze Complexity
Runtime 4 ms
Beats 55.73%
Memory 57.38 MB
Beats 38.39%

```

---

## まとめ

* **ジャッジ最適化版**：防御チェック排除 + null-proto辞書 + `in` → わずかに速く・やや省メモリになりやすい。
* **業務版**：`Map` + 最小限の検証で可読・保守優先。
  どちらも **Time O(n), Space O(n)**・Pure function・ESM/TS で問題要件を満たします。

